# 避免本地资源重复上传 效果演示

⚠ **警告：**
为了体现效果，本示例会清空本地存储，这有可能会给AnylearnSDK的正常使用带来一定影响，请谨慎执行

### 初始化SDK与Anylearn后端引擎连接

In [1]:
from anylearn.config import AnylearnConfig, init_sdk
from anylearn.storage.db import DB
init_sdk("http://192.168.10.22:31888", "xlearn", "123456")
AnylearnConfig.clear_workspace()
DB().force_init()

### 创建训练

In [2]:
import time
from anylearn.applications.quickstart import quick_train

In [3]:
st = time.time()

train_task1, algo1, dset1, project1 = quick_train(
    algorithm_dir="../../resources/cnn",
    dataset_dir="../../resources/fashion_mnist",
    entrypoint="python fashion_mnist.py",
    output="model",
    dataset_hyperparam_name="data-path",
    hyperparams={'batch-size': 256, 'epochs': 12},
    gpu_num=1,
)

ed = time.time()

print(f"TrainTask<name='{train_task1.name}' id='{train_task1.id}'>")
print(f"T = {round((ed - st) * 1000)} ms")

TrainTask<name='7dvce03t' id='TRAI2a96b95711eb9b152a4c200ec698'>
T = 13037 ms


#### 这里，本地算法和数据集均经过打包、校验、远程注册、上传等操作，耗时较长。

*注：本示例使用的算法文件大小为24KB，数据集文件大小为29MB*

### 以相同本地算法和数据集再次创建训练

In [4]:
st = time.time()

train_task2, algo2, dset2, project2 = quick_train(
    algorithm_dir="../../resources/cnn",
    dataset_dir="../../resources/fashion_mnist",
    entrypoint="python fashion_mnist.py",
    output="model",
    dataset_hyperparam_name="data-path",
    hyperparams={'batch-size': 256, 'epochs': 12},
    gpu_num=1,
)

ed = time.time()

print(f"TrainTask<name='{train_task2.name}' id='{train_task2.id}'>")
print(f"T = {round((ed - st) * 1000)} ms")

TrainTask<name='z3a4kprg' id='TRAI408cb95711eb9b152a4c200ec698'>
T = 1048 ms


#### 对于已经上传过的本地算法、数据集、模型等资源，Anylearn SDK会在本地做校验，并直接向远程后端请求资源的元信息，以此创建训练，避免了重复上传资源造成额外的网络和读写消耗。

#### 可以看到，第二次创建训练任务的耗时大幅降低。

*注：本示例使用的算法文件大小为24KB，数据集文件大小为29MB*

### 验证资源是否被复用

In [5]:
print(f"Same train task ? {train_task1.id == train_task2.id}")
print(f"Same algorithm ? {algo1.id == algo2.id}")
print(f"Same dataset ? {dset1.id == dset2.id}")

Same train task ? False
Same algorithm ? True
Same dataset ? True
